# レッスン 01  Predict-Next-Token

このチュートリアルでは、言語モデルの基本メカニズムである「次のトークン予測」（Predict-Next-Token）を深く理解していきます。ゼロから文字レベルの言語モデルを構築し、シェイクスピアの文体を学習させます。この実践プロジェクトを通じて、ニューラルネットワークがどのようにテキストパターンを学習し、特定のスタイルのテキストを生成するかを体験できます。

## Section 1 シェイクスピアテキストデータセット
言語モデルの構築を始める前に、まずトレーニングデータを準備する必要があります。ここでは古典的な「Tiny Shakespeare」データセット——シェイクスピアの全作品を含むテキストコレクションを使用します。

### なぜこのデータセットを選ぶのか？

* 独特なテキストスタイル：シェイクスピアの古英語の文体は特徴的で、モデルの学習効果を観察しやすい
* 適度なデータ量：約1MBのテキストで、高速なトレーニングと実験に適している
* 古典的な教育事例：言語モデル入門チュートリアルで広く使用されている

### データセットのダウンロード

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

このコマンドでGitHubからinput.txtファイルを現在のディレクトリにダウンロードします。

### テキストデータの読み込み

このコードは、テキストファイルを読み込み、データセットの規模を確認する方法を示しています。

In [ ]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

print("length of dataset in characters: ", len(text))

with open() ステートメント

* with はPythonのコンテキストマネージャーで、ファイルを自動的に閉じ、リソースリークを防ぎます
* エラーが発生しても、ファイルは正しく閉じられます
* "r": 読み取りモード（read mode）

### テキストの冒頭を確認
テキストの冒頭を確認任意のデータセットを処理する前に、実際の内容をプレビューすることは非常に重要なステップです。

In [ ]:
print(text[:200])

## Section 2 Pytorchをインストールする

PyTorchをインストールする前に、コンピュータのGPU構成を確認する必要があります。ニューラルネットワークのトレーニング時、GPUは計算速度を大幅に向上させます（通常CPUの100倍以上速い）。

### ステップ1：GPUとCUDAバージョンの確認

ターミナルまたはJupyter Notebookで以下のコマンドを実行します：

In [ ]:
!nvidia-smi